# Transform

## Import

In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
# Pfad
TEMPLATE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "_templates"))

In [3]:
# Lade die Pickle-Dateien
partei_df = pd.read_pickle(os.path.join(TEMPLATE_PATH, "parteien.pickle"))
wahlen = pd.read_pickle(os.path.join(TEMPLATE_PATH, "politbarometer.pickle"))

## Analyse

In [4]:
print(partei_df.shape, wahlen.shape)

(37948064, 4) (568, 8)


In [5]:
partei_df=partei_df.dropna()

In [6]:
partei_df.head()

,count,word,date,paper
1232,1,cdu,2021-04-01,sz
1771,1,cdu,2021-04-01,sz
2940,2,cdu,2021-04-01,zeit
3451,1,cdu,2021-04-01,zeit
6018,1,grünen,2021-04-01,faz


In [7]:
# Eine Kopie der Daten erstellen
wahlen = wahlen.copy()
wordcount = partei_df.copy()

## Wordcount df's erstellen

In diesem Code werden die gesuchten Parteinamen ab einem bestimmten Datum gefiltert, aggregiert und in verschiedene Zeitformate umgewandelt, um eine übersichtliche Auswertung zu ermöglichen.

In [8]:
# Einschränken auf Daten ab 2021-04-01 und Index zurücksetzen
wordcount = wordcount[wordcount['date'] >= '2021-04-01'].reset_index(drop=True)
wordcount = wordcount[wordcount["date"] <= "2024-12-01"].reset_index(drop=True)

# Gruppieren nach Datum, Wort und Zeitung sowie Aufsummieren der 'count'-Werte
wordcount = wordcount.groupby(['date', 'word', 'paper']).agg({'count': 'sum'}).reset_index()

# Konvertieren der 'date'-Spalte in ein Datumsformat
wordcount["date"] = pd.to_datetime(wordcount["date"])

# Hinzufügen einer Spalte für das Jahr und den Monat (Format 'YYYY-MM')
wordcount["month"] = wordcount["date"].dt.strftime('%Y-%m')

# Hinzufügen einer Spalte für das Quartal
wordcount["quarter"] = wordcount["date"].dt.to_period('Q')

# Ermitteln der Gesamtanzahl pro Wort
word_counts = wordcount.groupby(["word", "paper"])["count"].sum()


In [9]:
# Sicherstellen, dass die 'date'-Spalte als datetime-Objekt vorliegt
wordcount["date"] = pd.to_datetime(wordcount["date"])

# Woche hinzufügen: Startdatum der Woche berechnen
wordcount["week"] = wordcount["date"].dt.to_period("W").apply(lambda r: r.start_time)

# Ergebnis anzeigen
print(wordcount.head())

        date word         paper  count    month quarter       week
0 2021-04-01  afd           faz      2  2021-04  2021Q2 2021-03-29
1 2021-04-01  afd  tagesspiegel      1  2021-04  2021Q2 2021-03-29
2 2021-04-01  afd   uebermedien      5  2021-04  2021Q2 2021-03-29
3 2021-04-01  cdu      berliner      1  2021-04  2021Q2 2021-03-29
4 2021-04-01  cdu           dlf      6  2021-04  2021Q2 2021-03-29


## Wahlen (Politbarometer) df erstellen

In diesem Code werden die Barometerdaten auf einen bestimmten Zeitraum eingeschränkt, ihre Spalten vereinheitlicht und zusätzliche Zeitintervalle für die Analyse hinzugefügt.

In [10]:
# Konvertieren der 'date'-Spalte in ein Datumsformat
wahlen["date"] = pd.to_datetime(wahlen["date"])

# Daten auf Zeitraum vom 01.04.2021 bis 01.12.2024 beschränken
wahlen = wahlen[wahlen["date"] >= "2021-04-01"].reset_index(drop=True)
wahlen = wahlen[wahlen["date"] <= "2024-12-01"].reset_index(drop=True)

# Spaltennamen vereinheitlichen und neue Zeitspalten hinzufügen
wahlen.columns = wahlen.columns.str.lower()
wahlen["quarter"] = wahlen["date"].dt.to_period("Q").astype(str)
wahlen["month"] = wahlen["date"].dt.to_period("M").astype(str)


# Wandle alle spezifischen Spalten von float zu integer um
wahlen[['cdu', 'spd', 'grüne', 'fdp', 'linke', 'afd', 'bsw']] = wahlen[['cdu', 'spd', 'grüne', 'fdp', 'linke', 'afd', 'bsw']].fillna(0).astype(int)

# Sicherstellen, dass die 'date'-Spalte als datetime-Objekt vorliegt
wahlen["date"] = pd.to_datetime(wahlen["date"])

# Woche hinzufügen: Startdatum der Woche berechnen
wahlen["week"] = wahlen["date"].dt.to_period("W").apply(lambda r: r.start_time)

# Ergebnis anzeigen
print(wahlen.head())

# Länge jeder Spalte ausgeben
for key, value in wahlen.items():
    print(f'Länge von {key}:', len(value))


        date  cdu  spd  grüne  fdp  linke  afd  bsw quarter    month  \
0 2021-04-16   31   14     21    9      7   11    0  2021Q2  2021-04   
1 2021-05-07   25   14     26   10      7   11    0  2021Q2  2021-05   
2 2021-05-21   24   14     25   11      7   11    0  2021Q2  2021-05   
3 2021-06-10   28   15     22   10      7   11    0  2021Q2  2021-06   
4 2021-06-25   29   14     22   10      7   10    0  2021Q2  2021-06   

        week  
0 2021-04-12  
1 2021-05-03  
2 2021-05-17  
3 2021-06-07  
4 2021-06-21  
Länge von date: 72
Länge von cdu: 72
Länge von spd: 72
Länge von grüne: 72
Länge von fdp: 72
Länge von linke: 72
Länge von afd: 72
Länge von bsw: 72
Länge von quarter: 72
Länge von month: 72
Länge von week: 72


## json-Datei fuer d3 visualisierung


In [11]:
df = partei_df.copy()

# Datum umwandeln und sortieren
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(by=["word", "date"])

# Kumulative Summe berechnen
df["cumulative_count"] = df.groupby("word")["count"].cumsum()

# Datum in JSON-kompatibles Format umwandeln
df["date"] = df["date"].dt.strftime("%Y-%m-%d")

# Gruppieren und JSON-strukturierte Daten erstellen
grouped_data = df.groupby("word").apply(
    lambda group: group[["date", "cumulative_count"]].to_dict(orient="records")
).to_dict()

## Export

In [12]:
# Df's als pickle speichern
wahlen.to_pickle(os.path.join(TEMPLATE_PATH, "wahlen.pickle"))
word_counts.to_pickle(os.path.join(TEMPLATE_PATH, "kumuliert.pickle"))
wordcount.to_pickle(os.path.join(TEMPLATE_PATH, "wordcount.pickle"))

In [13]:
# JSON-Pfad fuer d3 Plot 
json_path = os.path.join(TEMPLATE_PATH, "cumulative_parteien.json")
 
# JSON-Datei speichern
with open(json_path, "w") as f:
    json.dump(grouped_data, f)

print(f"Kumulierte JSON-Daten gespeichert unter: {json_path}")

Kumulierte JSON-Daten gespeichert unter: c:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\_templates\cumulative_parteien.json
